### 1. Подключаем библиотеки

In [1]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import shutil
import random
from pathlib import Path

### 2. Подготовка датасета

##### Копируем изображения и создаем аннотации

In [2]:
def copy_images_with_annotations(src_dir, output_dir, class_idx, split, images, bbox_coords=(0.5, 0.5, 1.0, 1.0)):
    """Копирует изображения и создает аннотации"""
    for img_name in images:
        try:
            # Копируем изображение
            src_img = os.path.join(src_dir, img_name)
            dst_img = os.path.join(output_dir, 'images', split, img_name)
            shutil.copy2(src_img, dst_img)
            
            # Создаем YOLO аннотацию
            txt_name = os.path.splitext(img_name)[0] + '.txt'
            with open(os.path.join(output_dir, 'labels', split, txt_name), 'w') as f:
                f.write(f"{class_idx} {bbox_coords[0]} {bbox_coords[1]} {bbox_coords[2]} {bbox_coords[3]}\n")
                        
        except Exception as e:
            print(f"❌ Ошибка при обработке {img_name}: {e}")

##### Обрабатываем данные и проверяем результат

In [3]:
def process_single_tools(dataset_path, output_dir, classes):
    """Обрабатывает папки с отдельными инструментами"""
    print("\n=== Обработка отдельных инструментов ===")
    
    total_images = 0
    for class_idx, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)
        
        if not os.path.exists(class_path):
            print(f"⚠ Предупреждение: папка {class_path} не существует!")
            continue
            
        images = [f for f in os.listdir(class_path) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
        
        print(f"{class_name}: {len(images)} изображений")
        
        if len(images) == 0:
            print(f"⚠ Предупреждение: в папке {class_name} нет изображений!")
            continue
        
        # Разделяем на train/val
        if len(images) == 1:
            train_imgs = images
            val_imgs = []
        else:
            train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42, shuffle=True)
        
        print(f"  Train: {len(train_imgs)}, Val: {len(val_imgs)}")
        total_images += len(images)
        
        # Копируем изображения и создаем аннотации
        copy_images_with_annotations(class_path, output_dir, class_idx, 'train', train_imgs, 
                                   bbox_coords=(0.5, 0.5, 1.0, 1.0))  # Весь изображение
        copy_images_with_annotations(class_path, output_dir, class_idx, 'val', val_imgs,
                                   bbox_coords=(0.5, 0.5, 1.0, 1.0))
    
    print(f"✓ Обработано отдельных инструментов: {total_images} изображений")

def process_group_photos(dataset_path, output_dir, group_folders):
    """Обрабатывает групповые фотографии"""
    print("\n=== Обработка групповых фото ===")
    
    total_images = 0
    for group_folder in group_folders:
        group_path = os.path.join(dataset_path, group_folder)
        images = [f for f in os.listdir(group_path) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
        
        print(f"{group_folder}: {len(images)} изображений")
        
        if len(images) == 0:
            print(f"⚠ Предупреждение: в папке {group_folder} нет изображений!")
            continue
        
        total_images += len(images)
        
        # Для групповых фото используем все изображения для тренировки
        for img_name in images:
            try:
                # Копируем изображение в train
                src_img = os.path.join(group_path, img_name)
                dst_img = os.path.join(output_dir, 'images', 'train', img_name)
                shutil.copy2(src_img, dst_img)
                
                # Проверяем есть ли аннотация
                txt_name = os.path.splitext(img_name)[0] + '.txt'
                annotation_path = os.path.join(group_path, txt_name)
                
                if os.path.exists(annotation_path):
                    # Копируем существующую аннотацию
                    dst_txt = os.path.join(output_dir, 'labels', 'train', txt_name)
                    shutil.copy2(annotation_path, dst_txt)
                    print(f"  ✓ Добавлена аннотация для {img_name}")
                else:
                    # Создаем пустую аннотацию (нужно будет аннотировать вручную)
                    dst_txt = os.path.join(output_dir, 'labels', 'train', txt_name)
                    with open(dst_txt, 'w') as f:
                        # Пустой файл - нужно добавить bounding boxes вручную
                        pass
                    print(f"  ⚠ Создана пустая аннотация для {img_name} (требует ручной разметки)")
                    
            except Exception as e:
                print(f"❌ Ошибка при обработке группового фото {img_name}: {e}")
    
    print(f"✓ Обработано групповых фото: {total_images} изображений")
    
def check_dataset_stats(output_dir):
    """Проверяет статистику подготовленного датасета"""
    print("\n=== Статистика датасета ===")
    
    train_images = len(os.listdir(os.path.join(output_dir, 'images', 'train')))
    val_images = len(os.listdir(os.path.join(output_dir, 'images', 'val')))
    train_labels = len(os.listdir(os.path.join(output_dir, 'labels', 'train')))
    val_labels = len(os.listdir(os.path.join(output_dir, 'labels', 'val')))
    
    print(f"✓ Тренировочные изображения: {train_images}")
    print(f"✓ Валидационные изображения: {val_images}")
    print(f"✓ Тренировочные аннотации: {train_labels}")
    print(f"✓ Валидационные аннотации: {val_labels}")
    print(f"✓ Общее количество изображений: {train_images + val_images}")

##### Основная функция

In [4]:
def prepare_yolo_dataset(dataset_path, output_dir='yolo_dataset'):
    """
    Подготавливает датасет в формате YOLO
    """
    # Получаем все папки
    all_folders = sorted([d for d in os.listdir(dataset_path) 
                         if os.path.isdir(os.path.join(dataset_path, d))])
    
    print(f"Найдено папок: {len(all_folders)}")
    print("Все папки:", all_folders)
    
    # Создаем директории
    for split in ['train', 'val']:
        os.makedirs(os.path.join(output_dir, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(output_dir, 'labels', split), exist_ok=True)
    
    # Разделяем папки по типам
    tools_folders = [folder for folder in all_folders 
                    if 'групповые' not in folder.lower() 
                    and 'линейк' not in folder.lower()
                    and 'group' not in folder.lower()
                    and 'ruler' not in folder.lower()]
    
    group_folders = [folder for folder in all_folders 
                    if 'групповые' in folder.lower() 
                    or 'group' in folder.lower()]
    
    print(f"✓ Папки с инструментами: {len(tools_folders)}")
    print("Инструменты:", tools_folders)
    print(f"Групповые папки: {len(group_folders)}")
    
    # Сохраняем классы инструментов
    with open(os.path.join(output_dir, 'classes.txt'), 'w') as f:
        for i, class_name in enumerate(tools_folders):
            f.write(f"{class_name}\n")
    
    # Обрабатываем данные
    process_single_tools(dataset_path, output_dir, tools_folders)
    process_group_photos(dataset_path, output_dir, group_folders)
    
    # Проверяем результат
    check_dataset_stats(output_dir)
    
    return tools_folders

### 3. Создание конфигурации

In [5]:
def create_dataset_config(output_dir='yolo_dataset', output_yaml='dataset.yaml'):
    """
    Создает конфигурационный файл для датасета YOLO
    """
    # Читаем классы из файла
    classes_path = os.path.join(output_dir, 'classes.txt')
    if not os.path.exists(classes_path):
        print(f"Ошибка: файл {classes_path} не найден!")
        return None, None
    
    with open(classes_path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    
    # Создаем словарь с конфигурацией
    config = {
        'path': os.path.abspath(output_dir),
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(classes),
        'names': {i: class_name for i, class_name in enumerate(classes)}
    }
    
    # Сохраняем в YAML файл
    with open(output_yaml, 'w') as f:
        yaml.dump(config, f, default_flow_style=False, allow_unicode=True)
    
    print(f"✓ Создан конфиг файл: {output_yaml}")
    print(f"✓ Путь к данным: {os.path.abspath(output_dir)}")
    print(f"✓ Количество классов: {len(classes)}")
    return config, classes

### 4. Обучение модели

In [ ]:
def train_yolo_model(output_dir='yolo_dataset', model_size='s', epochs=100):
    """
    Обучение модели YOLOv8 с улучшенными параметрами
    """
    # Создаем конфигурационный файл
    config, classes = create_dataset_config(output_dir)
    
    if config is None:
        raise ValueError("Не удалось создать конфигурационный файл")
    
    # Загружаем предобученную модель
    print(f"🚀 Загрузка модели YOLOv8{model_size}.pt...")
    model = YOLO(f'yolov8{model_size}.pt')
    
    # Расширенные параметры обучения
    train_params = {
        'data': 'dataset.yaml',
        'epochs': epochs,
        'imgsz': 640,
        'batch': 16,
        'name': f'yolov8{model_size}_tools_detection_v2',
        'patience': 20,
        'optimizer': 'AdamW',
        'lr0': 0.001,
        'lrf': 0.01,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'augment': True,
        'hsv_h': 0.015,
        'hsv_s': 0.7,
        'hsv_v': 0.4,
        'degrees': 45,
        'translate': 0.1,
        'scale': 0.5,
        'shear': 0.0,
        'perspective': 0.0,
        'flipud': 0.0,
        'fliplr': 0.5,
        'mosaic': 1.0,
        'mixup': 0.1,
        'copy_paste': 0.1,
        'erasing': 0.4,
        'dropout': 0.1,
        'val': True,
        'save': True,
        'save_period': 10,
        #'device': 0,  # Использовать GPU если доступен
        'workers': 8,
        'single_cls': False,
        'verbose': True,
        'exist_ok': True
    }
    
    print("🎯 Начинаем обучение модели...")
    print(f"📊 Параметры обучения: {epochs} эпох, размер батча: 16")
    
    # Обучаем модель
    results = model.train(**train_params)
    
    return model, results

### 5. Экспорт в ONNX

In [7]:
def export_model_to_onnx(model, output_path='yolov8_tools.onnx'):
    """
    Экспорт модели в формат ONNX с оптимизацией
    """
    print("📤 Экспорт модели в ONNX...")
    model.export(format='onnx', imgsz=640, simplify=True, dynamic=True, opset=12)
    print(f"✅ Модель экспортирована в: {output_path}")

### 6. Оценка модели

In [8]:
def evaluate_model(model, data_path='yolo_dataset'):
    """
    Оценка модели на тестовых данных
    """
    print("📊 Оценка модели...")
    metrics = model.val(data=os.path.join(data_path, 'dataset.yaml'), split='val')
    return metrics


### 7. Тестирование

In [9]:
def predict_on_image(model, image_path, conf_threshold=0.25, iou_threshold=0.45):
    """
    Предсказание на одном изображении с улучшенной визуализацией
    """
    # Выполняем предсказание
    results = model(image_path, conf=conf_threshold, iou=iou_threshold, augment=True)
    
    # Визуализируем результаты
    for i, result in enumerate(results):
        # Рисуем bounding boxes с улучшенной визуализацией
        img = result.plot(line_width=2, font_size=1.0, conf=True, labels=True)
        
        # Создаем окно с фиксированным размером
        height, width = img.shape[:2]
        max_display_size = 1200
        if max(height, width) > max_display_size:
            scale = max_display_size / max(height, width)
            new_width = int(width * scale)
            new_height = int(height * scale)
            img = cv2.resize(img, (new_width, new_height))
        
        cv2.imshow('Инструменты - обнаружение', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        # Детальная информация об обнаруженных объектах
        print(f"\n🔍 Результаты обнаружения для изображения {i+1}:")
        if len(result.boxes) > 0:
            for j, box in enumerate(result.boxes):
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                bbox = box.xyxy[0].cpu().numpy()
                print(f"  Объект {j+1}: {model.names[class_id]} "
                      f"(уверенность: {confidence:.3f}) "
                      f"BBox: {bbox.astype(int)}")
        else:
            print("  Объекты не обнаружены")
    
    return results

### 8. Основная функция

In [10]:
def main():
    # Пути к данным
    dataset_path = '/data/vscode/HacatonAeroflot/Aeroflot-project/datasets/raw'
    output_dir = '/data/vscode/HacatonAeroflot/Aeroflot-project/yolo_dataset'
    
    try:
        # Проверяем существование пути
        if not os.path.exists(dataset_path):
            print(f"❌ Ошибка: путь {dataset_path} не существует!")
            return
        
        print("=" * 60)
        print("🛠️  СИСТЕМА ОБНАРУЖЕНИЯ ИНСТРУМЕНТОВ")
        print("=" * 60)
        
        # Шаг 1: Подготовка датасета
        print("\n📁 ШАГ 1: Подготовка датасета...")
        classes = prepare_yolo_dataset(dataset_path, output_dir)
        
        # Шаг 2: Создание конфигурации
        print("\n⚙️  ШАГ 2: Создание конфигурации...")
        config, classes = create_dataset_config(output_dir)
        
        if config is None:
            raise ValueError("Не удалось создать конфигурацию датасета")
        
        # Шаг 3: Обучение модели
        print("\n🎓 ШАГ 3: Обучение модели...")
        model, results = train_yolo_model(output_dir, model_size='s', epochs=100)
        
        # Шаг 4: Сохранение модели
        print("\n💾 ШАГ 4: Сохранение модели...")
        model.save('best_tools_detection.pt')
        print("✅ Модель сохранена как 'best_tools_detection.pt'")
        
        # Шаг 5: Экспорт в ONNX
        print("\n📤 ШАГ 5: Экспорт в ONNX...")
        export_model_to_onnx(model)
        
        # Шаг 6: Оценка модели
        print("\n📊 ШАГ 6: Оценка модели...")
        metrics = evaluate_model(model, output_dir)
        
        # Шаг 7: Тестирование
        print("\n🧪 ШАГ 7: Тестирование модели...")
        
        # Ищем тестовое изображение
        test_images_dir = os.path.join(output_dir, 'images', 'val')
        if os.path.exists(test_images_dir):
            test_images = os.listdir(test_images_dir)
            if test_images:
                test_image_path = os.path.join(test_images_dir, test_images[0])
                print(f"Тестируем на изображении: {test_images[0]}")
                predict_on_image(model, test_image_path)
            else:
                print("⚠ Тестовые изображения не найдены")
        else:
            print("⚠ Папка с тестовыми изображениями не найдена")
        
        print("\n" + "=" * 60)
        print("✅ ВСЕ ЭТАПЫ ЗАВЕРШЕНЫ УСПЕШНО!")
        print("=" * 60)
        
    except Exception as e:
        print(f"\n❌ КРИТИЧЕСКАЯ ОШИБКА: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

🛠️  СИСТЕМА ОБНАРУЖЕНИЯ ИНСТРУМЕНТОВ

📁 ШАГ 1: Подготовка датасета...
Найдено папок: 13
Все папки: ['1 Отвертка «-»', '10 Ключ рожковыйнакидной  ¾', '11 Бокорезы', '2 Отвертка «+»', '3 Отвертка на смещенный крест', '4 Коловорот', '5 Пассатижи контровочные', '6 Пассатижи', '7 Шэрница', '8 Разводной ключ', '9 Открывашка для банок с маслом', 'Групповые для тренировки', 'Инструменты с линейкой']
✓ Папки с инструментами: 11
Инструменты: ['1 Отвертка «-»', '10 Ключ рожковыйнакидной  ¾', '11 Бокорезы', '2 Отвертка «+»', '3 Отвертка на смещенный крест', '4 Коловорот', '5 Пассатижи контровочные', '6 Пассатижи', '7 Шэрница', '8 Разводной ключ', '9 Открывашка для банок с маслом']
Групповые папки: 1

=== Обработка отдельных инструментов ===
1 Отвертка «-»: 247 изображений
  Train: 197, Val: 50
❌ Ошибка при обработке DSCN2506.JPG: [Errno 1] Operation not permitted
❌ Ошибка при обработке DSCN2575.JPG: [Errno 1] Operation not permitted
❌ Ошибка при обработке DSCN4813.JPG: [Errno 1] Operation not perm

KeyboardInterrupt: 